In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-nov-2021/train.csv
/kaggle/input/tabular-playground-series-nov-2021/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

In [3]:
def evaluate_model(model, x, y):
    y_pred_prob = model.predict_proba(x)[:, 1]
    auc_roc = roc_auc_score(y, y_pred_prob)
    return {'auc_roc_curve' : auc_roc}

In [4]:
seed = 47

In [5]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv', sep=',')

# XGBoost Baseline

In [6]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

In [7]:
params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7463180392129056}


# Logistic Regression Baseline

In [8]:
model = LogisticRegression(random_state=seed, solver='liblinear')
model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7482452812440868}


# Feature Engineering

Here wee will experiment creating synthetic features using central tendency statistics.

In [9]:
geomean = lambda x, axis : np.exp(np.mean(np.log(x), axis=axis))
harmonic_mean = lambda x, axis : len(x) / np.sum(1.0/x, axis=axis) 

funcs = {'mean' : np.mean, 
         'std' : np.std, 
         'var' : np.var, 
         'geo_mean' : geomean, 
         'harmonic_mean' : harmonic_mean, 
         'median' : np.median}

<h3>Feature Engineering - XGBoost</h3>

In [10]:
results, names = list(), list()

params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

for key in funcs.keys():
    x_train = train_df.drop(['id', 'target'], axis=1)
    x_train[key] = funcs[key](x_train, axis=1)
    y_train = train_df['target']
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)       
    model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])
    
for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

>mean: 0.746157
>std: 0.746071
>var: 0.746071
>geo_mean: 0.746189
>harmonic_mean: 0.746146
>median: 0.746140


<h3>Feature Engineering - Logistic Regression</h3>

In [11]:
results, names = list(), list()

for key in funcs.keys():
    x_train = train_df.drop(['id', 'target'], axis=1)
    x_train[key] = funcs[key](x_train, axis=1)
    y_train = train_df['target']
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)       
    model = LogisticRegression(random_state=seed, solver='liblinear')
    
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])

for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

>mean: 0.748249
>std: 0.748224
>var: 0.746519
>geo_mean: 0.748245
>harmonic_mean: 0.721463
>median: 0.748217


# Grid Search Hyperparameters

In [12]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values

<h3>Grid Search Hyperparameters - XGBoost<h3>

In [13]:
params = {'n_estimators' : [10000, 11000],
          'max_depth' : [2, 3],
          'subsample' : [0.8, 0.9, 1.0],
          'eta' : [0.11, 0.12],
          'colsample_bytree' : [0.3, 0.4],
          'min_child_weight': [5],
          'gamma': [5]
         }

model = XGBClassifier(random_state=seed, tree_method='gpu_hist', predictor='gpu_predictor', use_label_encoder=False, verbosity=0)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
search = GridSearchCV(model, param_grid=params, scoring='roc_auc', refit='roc_auc', n_jobs=-1, cv=cv)

In [14]:
result = search.fit(x_train, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.7466454334143676
Best Hyperparameters: {'colsample_bytree': 0.3, 'eta': 0.11, 'gamma': 5, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 11000, 'subsample': 0.9}


# Ensemble Models

In [15]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

<h4>Fitting XGBoost</h4>

In [16]:
params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

xgboost_model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

xgboost_model.fit(x_train, y_train)
results = evaluate_model(xgboost_model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7463180392129056}


<h4>Fitting Logistic Regression</h4>

In [17]:
logistic_reg_model = LogisticRegression(random_state=seed, solver='liblinear')
logistic_reg_model.fit(x_train, y_train)
results = evaluate_model(logistic_reg_model, x_test, y_test)
print(results)

{'auc_roc_curve': 0.7482452812440868}


<h4>Ensembling both models</h4>

In [18]:
y_pred_prob_xgboost = xgboost_model.predict_proba(x_test)[:, 1]
y_pred_prob_logistic_reg = logistic_reg_model.predict_proba(x_test)[:, 1]
y_pred_prob = np.mean([y_pred_prob_xgboost, y_pred_prob_logistic_reg], axis=0)
auc_roc = roc_auc_score(y_test, y_pred_prob)
print({'auc_roc_curve' : auc_roc})

{'auc_roc_curve': 0.7485878280531844}


# Neural Nets

In [19]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

In [20]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=25)

2021-11-23 16:23:20.342950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 16:23:20.344150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 16:23:20.344800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 16:23:20.345582: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/25
3750/3750 [==============================] - 13s 3ms/step - loss: 0.6628 - auc: 0.6747
Epoch 2/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5902 - auc: 0.7410
Epoch 3/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5854 - auc: 0.7426
Epoch 4/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5824 - auc: 0.7444
Epoch 5/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5807 - auc: 0.7460
Epoch 6/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5808 - auc: 0.7453
Epoch 7/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5797 - auc: 0.7464
Epoch 8/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5788 - auc: 0.7469
Epoch 9/25
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5785 - auc: 0.7469
Epoch 10/25
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5780 - auc: 0.7469
Epoch 11/25
3750/37

In [21]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5714 - auc: 0.7508
0.7508249282836914


In [22]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=35)

Epoch 1/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.8525 - auc_1: 0.5303
Epoch 2/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6631 - auc_1: 0.6417
Epoch 3/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.6361 - auc_1: 0.6979
Epoch 4/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.6178 - auc_1: 0.7212
Epoch 5/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.6044 - auc_1: 0.7335
Epoch 6/35
3750/3750 [==============================] - 13s 4ms/step - loss: 0.5959 - auc_1: 0.7393
Epoch 7/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5903 - auc_1: 0.7429
Epoch 8/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5859 - auc_1: 0.7452
Epoch 9/35
3750/3750 [==============================] - 13s 3ms/step - loss: 0.5831 - auc_1: 0.7466
Epoch 10/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5811 - auc_1: 0.7474

In [23]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 8s 2ms/step - loss: 0.5712 - auc_1: 0.7511
0.7510676383972168


In [24]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=1024, epochs=40)

Epoch 1/40
469/469 [==============================] - 2s 4ms/step - loss: 1.4060 - auc_2: 0.5054
Epoch 2/40
469/469 [==============================] - 2s 4ms/step - loss: 0.7119 - auc_2: 0.5088
Epoch 3/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6957 - auc_2: 0.5135
Epoch 4/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6914 - auc_2: 0.5273
Epoch 5/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6861 - auc_2: 0.5491
Epoch 6/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6778 - auc_2: 0.5906
Epoch 7/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6636 - auc_2: 0.6403
Epoch 8/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6497 - auc_2: 0.6730
Epoch 9/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6406 - auc_2: 0.6904
Epoch 10/40
469/469 [==============================] - 2s 4ms/step - loss: 0.6331 - auc_2: 0.7025
Epoch 11/40
469/469 [========

In [25]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5745 - auc_2: 0.7482
0.7482479810714722


In [26]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=35)

Epoch 1/35
3750/3750 [==============================] - 12s 3ms/step - loss: 1.0391 - auc_3: 0.5143
Epoch 2/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6746 - auc_3: 0.6082
Epoch 3/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.6408 - auc_3: 0.6889
Epoch 4/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.6189 - auc_3: 0.7193
Epoch 5/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.6046 - auc_3: 0.7329
Epoch 6/35
3750/3750 [==============================] - 14s 4ms/step - loss: 0.5957 - auc_3: 0.7396
Epoch 7/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5902 - auc_3: 0.7429
Epoch 8/35
3750/3750 [==============================] - 12s 3ms/step - loss: 0.5860 - auc_3: 0.7453
Epoch 9/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5831 - auc_3: 0.7465
Epoch 10/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5813 - auc_3: 0.7471

In [27]:
results = model.evaluate(x_test, y_test)[1]
print(results)

3750/3750 [==============================] - 9s 2ms/step - loss: 0.5718 - auc_3: 0.7510
0.7510152459144592


In [28]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='selu'))
model.add(tf.keras.layers.Dense(128, activation='selu'))
model.add(tf.keras.layers.Dense(64, activation='selu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=35)
results = model.evaluate(x_test, y_test)[1]
print(results)

Epoch 1/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.6742 - auc_4: 0.6942
Epoch 2/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5976 - auc_4: 0.7372
Epoch 3/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5880 - auc_4: 0.7428
Epoch 4/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5831 - auc_4: 0.7448
Epoch 5/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5803 - auc_4: 0.7458
Epoch 6/35
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5785 - auc_4: 0.7472
Epoch 7/35
3750/3750 [==============================] - 13s 4ms/step - loss: 0.5780 - auc_4: 0.7473
Epoch 8/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5779 - auc_4: 0.7468
Epoch 9/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5768 - auc_4: 0.7481
Epoch 10/35
3750/3750 [==============================] - 10s 3ms/step - loss: 0.5757 - auc_4: 0.7484

# Submission

In [29]:
x_train = train_df.drop(['id', 'target'], axis=1)
y_train = train_df['target']

In [30]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=25)

Epoch 1/25
4688/4688 [==============================] - 18s 4ms/step - loss: 0.8636 - auc_5: 0.5303
Epoch 2/25
4688/4688 [==============================] - 17s 4ms/step - loss: 0.6552 - auc_5: 0.6599
Epoch 3/25
4688/4688 [==============================] - 17s 4ms/step - loss: 0.6234 - auc_5: 0.7141
Epoch 4/25
4688/4688 [==============================] - 17s 4ms/step - loss: 0.6049 - auc_5: 0.7324
Epoch 5/25
4688/4688 [==============================] - 17s 4ms/step - loss: 0.5939 - auc_5: 0.7405
Epoch 6/25
4688/4688 [==============================] - 16s 3ms/step - loss: 0.5873 - auc_5: 0.7443
Epoch 7/25
4688/4688 [==============================] - 18s 4ms/step - loss: 0.5833 - auc_5: 0.7460
Epoch 8/25
4688/4688 [==============================] - 23s 5ms/step - loss: 0.5804 - auc_5: 0.7475
Epoch 9/25
4688/4688 [==============================] - 17s 4ms/step - loss: 0.5787 - auc_5: 0.7479
Epoch 10/25
4688/4688 [==============================] - 17s 4ms/step - loss: 0.5771 - auc_5: 0.7496

In [31]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv', sep=',')
x_test = test_df.drop(['id'], axis=1)
target = model.predict(x_test).squeeze()
ids = test_df['id'].values
submission = pd.DataFrame({'id' : ids, 'target' : target})

In [32]:
submission.head()

,id,target
0,600000,0.747956
1,600001,0.799313
2,600002,0.798621
3,600003,0.247537
4,600004,0.784533


In [33]:
submission.to_csv('submission.csv', index=False)